In [1]:
import os
from datetime import datetime
from dotenv import load_dotenv

#--------Google Drive Integration--------#
# from google.colab import drive, userdata
# This gives Colab access to your files in Google Drive.
# drive.mount('/content/drive')
# 'GITHUB_USERNAME' and 'GITHUB_TOKEN' saved as secrets in Colab.
# GITHUB_USERNAME = userdata.get('GITHUB_USERNAME')
# GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
# REPOSITORY_NAME = 'PyNucleus-Model' # Your repository name
# NOTEBOOK_DRIVE_PATH = "/content/drive/MyDrive/PyNucleus Project/Capstone Project.ipynb"


#--------Cursor Integration--------#
# Load environment variables from .env file
load_dotenv()

# Get GitHub credentials from environment variables
GITHUB_USERNAME = os.getenv('GITHUB_USERNAME')
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

# Print to verify the variables are loaded (remove this in production)
print(f"Username: {GITHUB_USERNAME}")
print(f"Token: {GITHUB_TOKEN[:4]}...") # Only print first 4 chars of token for security

# Repository information
REPOSITORY_NAME = 'PyNucleus-Model'
NOTEBOOK_REPO_FILENAME = "Capstone Project.ipynb"
LOG_FILENAME = "update_log.txt"

# Pull latest changes from GitHub
print("Pulling latest changes from GitHub...")
!git pull https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git main

print("Repository is up to date!")

# Add a new line to your log file with the current date and time.
log_message = f"Notebook saved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
with open(LOG_FILENAME, "a") as f:
    f.write(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Your update message here")

Username: Saytor20
Token: ghp_...
Pulling latest changes from GitHub...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 28 (delta 5), reused 5 (delta 5), pack-reused 23 (from 1)
Unpacking objects: 100% (28/28), 3.30 KiB | 225.00 KiB/s, done.
From https://github.com/Saytor20/PyNucleus-Model
 * branch            main       -> FETCH_HEAD
Updating a4a4739..59618e0
error: Your local changes to the following files would be overwritten by merge:
	Capstone Project.ipynb
	update_log.txt
Please commit your changes or stash them before you merge.
Aborting
Repository is up to date!


In [2]:
# # Clone the Repository
# repo_path = f'/content/{REPOSITORY_NAME}'
# !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git {repo_path}

# # Change the current working directory to the repository
# # All subsequent commands will run from inside the repo folder.
# os.chdir(repo_path)

# # Add a new line to your log file with the current date and time.
# log_message = f"Notebook saved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
# with open(LOG_FILENAME, "a") as f:
#     f.write(log_message + "\n")
# print(f"Updated '{LOG_FILENAME}'")

# # Copy the latest version of notebook from Drive into the cloned repo.
# !cp "{NOTEBOOK_DRIVE_PATH}" "{NOTEBOOK_REPO_FILENAME}"
# print(f"Copied '{NOTEBOOK_REPO_FILENAME}' from Google Drive.")

# # Git identity for commenting
# !git config user.name "{GITHUB_USERNAME}"
# !git config user.email "{GITHUB_USERNAME}@users.noreply.github.com"

In [3]:
# # --- Installation Cell (Run this ONCE per session) ---
# print("Installing all required packages...")

# %pip install -q langchain langchain-core langchain-community langchain-text-splitters

# %pip install -q "unstructured[local-inference]"

# # LLM & ML Libraries
# %pip install -q transformers accelerate bitsandbytes torch sentence-transformers

# # Vector Stores
# %pip install -q chromadb faiss-cpu # faiss-gpu if you have a Pro Colab with a good GPU

# # Data Handling & Utilities
# %pip install -q pandas numpy tqdm PyYAML

# print("All packages installed successfully.")

In [4]:
# --- Core Utilities & File Handling ---
import getpass
import yaml # Make sure PyYAML is installed
from pathlib import Path

# --- Data Handling & Progress Bars ---
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- Document Loading ---
# UnstructuredFileLoader is now in langchain_community
from langchain_community.document_loaders import UnstructuredFileLoader
# For future testing of more specific loaders:
# from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader

# --- OCR Libraries ---
# No direct imports needed here if using UnstructuredFileLoader with local-inference,
# as it handles OCR internally.
# You would only import these for a manual OCR process:
# import pytesseract
# from pdf2image import convert_from_path
# from PIL import Image

# --- Core ML & LLM Libraries ---
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes
import accelerate # Often used with transformers

# --- Vector Stores & Embeddings ---
# Using ChromaDB as the primary vector store.
import chromadb
from langchain_community.vectorstores import Chroma # Chroma is now in langchain_community
from langchain_community.embeddings import HuggingFaceEmbeddings # Embeddings are also in langchain_community
# For future testing of a high-performance alternative:
# import faiss
# from langchain_community.vectorstores import FAISS

# --- RAG/Agent Frameworks ---
# Core LangChain components
import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# For future testing of an alternative RAG-focused toolkit:
# from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


# **Data Ingestion and Preprocessing for RAG**

# **Data Scrapping**

# This is the last cell of the code

In [5]:
# Update log file with your message
log_message = f"Notebook saved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
with open(LOG_FILENAME, "a") as f:
    f.write(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Your update message here")

# Push changes to GitHub
!git add {LOG_FILENAME}  
!git commit -m f"Update: {update_message}"
!git push https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git main

print("Changes pushed to GitHub successfully!")

[main 29138c1] fUpdate: {update_message}
 1 file changed, 25 insertions(+), 2 deletions(-)
To https://github.com/Saytor20/PyNucleus-Model.git
 ! [rejected]        main -> main (non-fast-forward)
error: failed to push some refs to 'https://github.com/Saytor20/PyNucleus-Model.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. If you want to integrate the remote changes,
hint: use 'git pull' before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
Changes pushed to GitHub successfully!
